# GiroE GARMIN fit Analysis

In [3]:
import pandas as pd
import fitdecode, os, json
from joblib import Parallel, delayed

## Utils

In [2]:
DATADIR = "GarminRawData"


def decode_fit(dirName, fName):
    with fitdecode.FitReader(os.path.join(dirName, fName)) as fit:
        for frame in fit:
            # The yielded frame object is of one of the following types:
            # * fitdecode.FitHeader (FIT_FRAME_HEADER)
            # * fitdecode.FitDefinitionMessage (FIT_FRAME_DEFINITION)
            # * fitdecode.FitDataMessage (FIT_FRAME_DATA)
            # * fitdecode.FitCRC (FIT_FRAME_CRC)

            if frame.frame_type == fitdecode.FIT_FRAME_DATA:
                # Here, frame is a FitDataMessage object.
                # A FitDataMessage object contains decoded values that
                # are directly usable in your script logic.
                print(frame.name)
                if frame.name == "record":
                    print("record")
                else:
                    print("else")

## Files descriptions

In [3]:
# for f in $(ls GarminRawData |grep -v .zip)
# do 
#     for fitf in $(ls GarminRawData/$f)
#         do
#         echo $fitf
#         done
# done

In [4]:
# for datadir in [os.path.join(DATADIR,d) for d in os.listdir(DATADIR) if not d.endswith('.zip') and not d.startswith('.')]:
#     for file in os.listdir(datadir):
#         print(os.path.join(datadir,file))
#         decode_fit(datadir, file)
#         break
#     break

In [8]:
def parse(f):
    try:
        json_file = f[0]
        df = pd.read_json(json_file)
        record_df = df.loc[(df["frame_type"] == "data_message") & (df.name == "record")]
        values_df = pd.concat(
            [
                pd.DataFrame.from_records(l)[["name", "value"]]
                .set_index("name")
                .transpose()
                for l in record_df.fields
            ]
        )
        if values_df.shape[0] !=0:
            values_df.to_csv(f"{json_file[:-8]}csv")
            values_df["timestamp"] = pd.to_datetime(values_df["timestamp"])
            # values_df.to_excel(f"{json_file[:-8]}xlsx")
            stats = {}
            stats["data_filename"] = json_file
            stats["t_start"] = values_df.timestamp.min()
            stats["t_stop"] = values_df.timestamp.max()
            stats["duration_sec"] = stats["t_stop"] - stats["t_start"]

            stats["t_start"] = stats["t_start"].isoformat()
            stats["t_stop"] = stats["t_stop"].isoformat()
            stats["duration_sec"] = stats["duration_sec"].total_seconds()

            stats.update(
                pd.DataFrame.from_records(
                    df.loc[
                        (df.frame_type == "data_message") & (df.name == "file_id")
                    ].fields.values[0]
                )[["name", "value"]]
                .set_index("name")
                .to_dict()["value"]
            )
            with open(f"{json_file[:-9]}-stat.json", "w") as f:
                json.dump(stats, f, indent=2)
    except ValueError:
        print(f"No points in {json_file}")
    except KeyError:
        print(f"Empty file {json_file}")
   


Parallel(n_jobs=10)(
    delayed(parse)(f)
    for f in pd.read_csv(
        "/Users/paolo/PycharmProjects/most-garmin-grabber/files.csv", header=None
    ).values
)

No points in GarminRawData/3818/2024-05-05-15-58-25.fit.json
No points in GarminRawData/4642/2024-05-25-15-15-05.fit.json


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

## Read stat files

In [12]:
pd.read_csv('stat.csv', header=None).values[0]

array(['./GarminRawData/4141/2024-05-26-12-08-55-stat.json'], dtype=object)